In [1]:
!pip install -q transformers torch


In [2]:
from transformers import pipeline


In [3]:
ner_pipeline = pipeline(
    "ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    grouped_entities=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [4]:
resume_text = """
Sneha S
Email: sneha@gmail.com
Phone: 9876543210

Education:
Bachelor of Technology in Computer Science
PES University, India

Experience:
Software Intern at Google
Worked on NLP-based text classification systems.

Location:
Bangalore, India
"""


In [6]:
entities = ner_pipeline(resume_text)
entities


[{'entity_group': 'ORG',
  'score': np.float32(0.77490246),
  'word': 'Sneha S',
  'start': 1,
  'end': 8},
 {'entity_group': 'ORG',
  'score': np.float32(0.5273906),
  'word': '##ne',
  'start': 17,
  'end': 19},
 {'entity_group': 'ORG',
  'score': np.float32(0.79228914),
  'word': 'Computer Science PES University',
  'start': 88,
  'end': 119},
 {'entity_group': 'LOC',
  'score': np.float32(0.99360585),
  'word': 'India',
  'start': 121,
  'end': 126},
 {'entity_group': 'ORG',
  'score': np.float32(0.99503577),
  'word': 'Google',
  'start': 159,
  'end': 165},
 {'entity_group': 'MISC',
  'score': np.float32(0.8660177),
  'word': 'NLP',
  'start': 176,
  'end': 179},
 {'entity_group': 'LOC',
  'score': np.float32(0.9993839),
  'word': 'Bangalore',
  'start': 226,
  'end': 235},
 {'entity_group': 'LOC',
  'score': np.float32(0.99910057),
  'word': 'India',
  'start': 237,
  'end': 242}]

In [7]:
extracted = {
    "Name": [],
    "Organizations": [],
    "Locations": []
}

for entity in entities:
    if entity["entity_group"] == "PER":
        extracted["Name"].append(entity["word"])
    elif entity["entity_group"] == "ORG":
        extracted["Organizations"].append(entity["word"])
    elif entity["entity_group"] == "LOC":
        extracted["Locations"].append(entity["word"])

extracted


{'Name': [],
 'Organizations': ['Sneha S',
  '##ne',
  'Computer Science PES University',
  'Google'],
 'Locations': ['India', 'Bangalore', 'India']}

In [9]:
# -------- Rule-Based Name Extraction Fix --------

lines = [line.strip() for line in resume_text.split("\n") if line.strip()]

# Heuristic:
# 1. Name is usually in the first 1–2 lines
# 2. Contains only alphabets and spaces
# 3. Length is reasonable (not too long)

def extract_name_rule_based(lines):
    for line in lines[:3]:
        if (
            line.replace(" ", "").isalpha()
            and 2 <= len(line.split()) <= 4
            and len(line) < 40
        ):
            return line
    return "Not Found"

rule_based_name = extract_name_rule_based(lines)

print("Rule-Based Name Guess:")
print(rule_based_name)


Rule-Based Name Guess:
Sneha S


In [14]:
print("Extracted Resume Information\n")

final_name = (
    rule_based_name
    if len(extracted["Name"]) == 0
    else list(set(extracted["Name"]))[0]
)

print("Name:")
print(final_name)

print("\nOrganizations:")
clean_orgs = {
    org for org in extracted["Organizations"]
    if not org.startswith("##")
    and len(org) > 2
    and org != final_name
}

print(clean_orgs)

print("\nLocations:")
print(set(extracted["Locations"]))


Extracted Resume Information

Name:
Sneha S

Organizations:
{'Computer Science PES University', 'Google'}

Locations:
{'Bangalore', 'India'}
